<a href="https://colab.research.google.com/github/Prograndma/detect_ai/blob/main/Detecting_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting AI
This is a general way to detect text that has been generated by a transformer based LLM. This approach is nice becaues it runs very quickly, and doesn't require any sort of fine-tuning. Using gpt2, I'm able to get good results, even on generated text from gpt4o, and even claude and gemini. Go ahead, try it!

In [1]:
# @title Change the model and tokenizer used here, any pre-trained LLM should work. Here I use GPT2.
%%capture
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = 'gpt2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


In [2]:
# @title The algorithm. Read this simple code to see what I'm doing.
if device == 'cpu':
    print("For better performance, use a GPU")
def prob_per_token(sequence: str, tokenizer, model, device="cuda"):
    inputs = tokenizer(sequence.strip(), return_tensors="pt", truncation=True).to(device)

    # Compute logits without gradient calculations
    with torch.no_grad():
        logits = model(**inputs).logits

    # Compute probabilities
    probs = torch.softmax(logits, dim=-1)

    # Extract token IDs and calculate probabilities

    seq_token_ids = inputs["input_ids"][0]
    seq_probs = probs[0, torch.arange(seq_token_ids.shape[0]), seq_token_ids]
    token_likelihoods = seq_probs.tolist()

    if len(token_likelihoods) == 0:
        print("There was an error with the model")
        return 0.0
    else:
        avg_prob = sum(token_likelihoods) / len(token_likelihoods)
        return avg_prob

In [3]:
# @title Importing a UI
%%capture
!pip install ipywidgets

from IPython.display import display
import ipywidgets as widgets

In [4]:
# @title Run this (and every cell above it) to test out my AI detector!
# Create a text area widget
text_area = widgets.Textarea(
    placeholder='Paste your text here...',
    description='Input:',
    layout=widgets.Layout(width='50%', height='300px', background='white'),
    style={'description_width': 'initial'}
)
text_area.layout.border = '3px solid white'
# Create a submit button
submit_button = widgets.Button(
    description='Submit',
    button_style='success',  # Green button
)

# Create a label for messages
message_label = widgets.Label(value='', layout=widgets.Layout(height='30px'))
lower_threshold = 0.003821577792116015
upper_threshold = 0.03426106060988655

# Function to evaluate the input
def evaluate_input(text):
    # Replace this logic with your custom evaluation
    score = prob_per_token(text, tokenizer, model, device)
    return (score < lower_threshold) or (score > upper_threshold)

# Function to handle button click
def on_button_click(b):
    if evaluate_input(text_area.value):
        text_area.layout.background = 'lightgreen'
        text_area.layout.border = '3px solid lightgreen'
        message_label.value = "✅ Looks human written to me!"
    else:
        text_area.layout.background = 'lightcoral'
        text_area.layout.border = '3px solid lightcoral'
        message_label.value = "❌ Text might be generated."

# Function to reset the background and message when text is changed
def on_text_change(change):
    if len(text_area.value.split(' ')) > 800:
      text_area.layout.background = 'yellow'
      text_area.layout.border = '3px solid yellow'
      message_label.value = "⚠️ Your input is too long! It will be truncated."
    else:
      text_area.layout.background = 'white'  # Neutral background
      text_area.layout.border = '3px solid white'
      message_label.value = ''  # Clear the message

# Attach the events
submit_button.on_click(on_button_click)
text_area.observe(on_text_change, names='value')

# Display the widgets
display(text_area, submit_button, message_label)

Textarea(value='', description='Input:', layout=Layout(border='3px solid white', height='300px', width='50%'),…

Button(button_style='success', description='Submit', style=ButtonStyle())

Label(value='', layout=Layout(height='30px'))